In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

details_2005 = pd.read_csv("./StormEvents_details-ftp_v1.0_d2005_c20170717.csv")
locations_2005 = pd.read_csv("./StormEvents_locations-ftp_v1.0_d2005_c20170717.csv")
#import 2014-2019

In [2]:
details_2005.columns = [x.lower() for x in list(details_2005.columns)]

In [3]:
locations_2005.columns = [x.lower() for x in list(locations_2005.columns)]

In [4]:
drop_columns = []

for i in details_2005.isna().sum().index:
    if (details_2005.isna().sum() > 40_000)[i]:
        drop_columns.append(i)

In [5]:
details_2005 = details_2005.drop(drop_columns, axis = 1)

In [6]:
merged = pd.merge(locations_2005, details_2005, on = "event_id")
merged.shape

(57267, 51)

In [7]:
def convert_letters(string):
    try:
        if string.find("K") > 0:
            return float(string.replace("K",""))*1_000
        elif string.find("M") > 0:
            return float(string.replace("M",""))*1_000_000
        elif string.find("B") > 0:
            return float(string.replace("B",""))*1_000_000_000
        else:
            return np.nan
    except:
        print (string)

In [8]:
def find_storm(lat, 
               long, 
               dist, 
               merged_dataframe,
               metric = "median"):
    
    merged_df = merged_dataframe.copy()
    
    merged_df_lat_mask_upper = merged_df["latitude"] <= (lat + dist)
    merged_df_lat_mask_lower = merged_df["latitude"] >= (lat - dist)
    merged_df_long_mask_upper = merged_df["longitude"] <= (long + dist)
    merged_df_long_mask_lower = merged_df["longitude"] >= (long - dist)
    
    merged_df["damage_property"] = merged_df["damage_property"].apply(lambda x: convert_letters(x) if type(x) != float else np.nan)
    
    data = merged_df[merged_df_lat_mask_lower & merged_df_lat_mask_upper & merged_df_long_mask_lower & merged_df_long_mask_upper]
    median = data.groupby("event_type").median()[["range","deaths_direct", "damage_property"]]
    median["count"] = data.groupby("event_type").count()["state"].values
    mean = data.groupby("event_type").mean()[["range", "deaths_direct", "damage_property"]]
    mean["count"] = data.groupby("event_type").count()["state"].values
    
    if metric == "mean":
        return mean.sort_values("count", ascending = False)
    else:
        return median.sort_values("count", ascending = False)
    
    
    

In [9]:
df = find_storm(41, -83, 5,merged, metric = "mean")

In [10]:
df

,range,deaths_direct,damage_property,count
event_type,,,,
Thunderstorm Wind,4.145594,0.001402,1.641422e+04,2854
Hail,3.721127,0.000000,7.099448e+03,1075
Marine Thunderstorm Wind,4.807018,0.000000,8.269231e+03,773
Flood,NaN,0.008772,3.281597e+05,342
Flash Flood,4.740000,0.007463,2.316842e+05,268
Tornado,2.948454,0.410256,2.997484e+06,117
Marine Hail,7.235294,0.000000,NaN,34
Waterspout,2.400000,0.000000,NaN,11
Lightning,1.500000,0.100000,1.000000e+04,10
